# Optical recordings processing with pyCardiac

This notebook demonstrates general pipeline of processing and analysis of Action Potential optical recordings.

Please go through this notebook step by step. 

## Content

* 1. <a href="#section1" > Data loading </a>

* 2. <a href="#section2" > Processing </a>
    * 2.1. <a href="#section2.1" > Fourier filtration</a>
    * 2.2. <a href="#section2.2" > Baseline removal </a>
    * 2.3. <a href="#section2.3" > Binning (spatial filtration) </a>
    * 2.4. <a href="#section2.4" > Rescaling (normalizing) </a>
    * 2.5. <a href="#section2.5" > Ensemble averaging </a>
    * 2.6. <a href="#section2.6" > Transform to phase </a>
    
* 3.<a href="#section3" > Mapping </a>
    * 3.1. <a href="#section3.1" > APD and Alternance maps </a>
    * 3.2. <a href="#section3.2" > Activation time map </a>
    * 3.3. <a href="#section3.3" > Phase singularity points (PS) </a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import pyCardiac as pc
from pyCardiac.rhythm import *
from pyCardiac.signal.analysis import phase_singularity_detection

<a id='section1'></a>
## Data loading

In [ ]:
data_raw_filename = "./source/sample_optical_recordings.npy"
data_raw = np.load(data_raw_filename)

<a id='section2'></a>
## Processing

In [ ]:
data = data_raw.copy()
mask = np.loadtxt("./source/sample_optical_recordings_mask.txt").astype(np.bool)
x, y, t = 80, 60, 15

In [ ]:
plt.figure(figsize = (10, 3.5))

plt.subplot(1, 2, 1)
frame = np.ma.masked_where(~mask, data[:, :, t])
plt.imshow(frame, cmap="jet")
plt.plot(x, y, '*w', ms = 10)
plt.colorbar()
plt.title("data, t = {t}".format(t = t))

plt.subplot(1, 2, 2)
plt.imshow(mask, cmap = "Greys_r")
plt.plot(x, y, '*k', ms = 10)
plt.title("mask")

plt.show()

plt.figure(figsize = (10, 3.5))
plt.plot(data[y, x, :])
plt.title("signal: x = {x}, y = {y}".format(x=x, y=y))
plt.show()

<a id='section2.1'></a>
### Fourier filtration

In [ ]:
kwargs = {'fs' : 1000, # sampling frequency (Hz)
          'lp_freq' : 100, # lowpass frequency (Hz)
          'hp_freq' : None, # highpass frequency (Hz)
          'bs_freqs' : [60, ], # bandstop frequency (Hz)
          'trans_width' : 2, # width of transition region between bands (Hz)
          'band_width' : 2, # width of bandstop in (Hz)
         }
%time data_filtered = fourier_filter(data, **kwargs)

In [ ]:
plt.figure(figsize = (10, 3.5))
plt.subplot(1, 2, 1)
frame = np.ma.masked_where(~mask, data[:, :, t])
plt.imshow(frame, cmap = "jet")
plt.plot(x, y, '*w', ms = 10)
plt.colorbar()
plt.title("data, t = {t}".format(t = t))

plt.subplot(1, 2, 2)
frame = np.ma.masked_where(~mask, data_filtered[:, :, t])
plt.imshow(frame, cmap = "jet")
plt.plot(x, y, '*w', ms = 10)
plt.colorbar()
plt.title("data filtered, t = {t}".format(t = t))

plt.show()

plt.figure(figsize = (10, 3.5))
plt.plot(data[y, x, :],
         label = "data", color = "C0")
plt.plot(data_filtered[y, x, :],
         label = "data filtered", color = "C1")
plt.title("Signal: x = {x}, y = {y}".format(x=x, y=y))
plt.legend()
plt.show()

<a id='section2.2'></a>
### Baseline removal

In [ ]:
#Linear detrending
%time data_detrended = remove_baseline(data_filtered)

# Asymmetric-Least-Squares Method (slow but more powerful)
# uncomment to use
#niter = 2
#%time data_detrended = remove_baseline(data_filtered, method_name="least_squares", niter = niter)

In [ ]:
plt.figure(figsize = (10, 3.5))
plt.subplot(1, 2, 1)
frame = np.ma.masked_where(~mask, data_filtered[:, :, t])
plt.imshow(frame, cmap = "jet")
plt.plot(x, y, '*w', ms = 10)
plt.colorbar()
plt.title("data filtered, t = {t}".format(t = t))

plt.subplot(1, 2, 2)
frame = np.ma.masked_where(~mask, data_detrended[:, :, t])
plt.imshow(frame, cmap = "jet")
plt.plot(x, y, '*w', ms = 10)
plt.colorbar()
plt.title("data detrended, t = {t}".format(t = t))

plt.show()


fig, ax1 = plt.subplots(figsize = (10, 3.5))
ax1.plot(data_filtered[y, x, :], color = "C1")
ax1.set_title("Signal: x = {x}, y = {y}".format(x=x, y=y))
ax1.set_ylabel("data filtered", color='C1')

ax2 = ax1.twinx()
ax2.plot(data_detrended[y, x, :], color = "C2")
ax2.set_ylabel("data detrended", color='C2')
plt.show()

<a id='section2.3'></a>
### Binning (spatial filtration)

In [ ]:
%time data_binned = binning(data_detrended, 9, "gaussian", mask)

In [ ]:
plt.figure(figsize = (10, 3.5))

plt.subplot(1, 2, 1)
frame = np.ma.masked_where(~mask, data_detrended[:, :, t])
plt.imshow(frame, cmap = "jet")
plt.plot(x, y, '*w', ms = 10)
plt.colorbar()
plt.title("data detrended, t = {t}".format(t = t))

plt.subplot(1, 2, 2)
frame = np.ma.masked_where(~mask, data_binned[:, :, t])
plt.imshow(frame, cmap = "jet")
plt.plot(x, y, '*w', ms = 10)
plt.colorbar()
plt.title("data binned, t = {t}".format(t = t))

plt.show()

plt.figure(figsize = (10, 3.5))
plt.plot(data_detrended[y, x, :],
         label = "data detrended", color = "C2")
plt.plot(data_binned[y, x, :],
         label = "data binned", color = "C3")
plt.title("Signal: x = {x}, y = {y}".format(x=x, y=y))
plt.legend()
plt.show()

<a id='section2.4'></a>
### Rescaling (normalizing)

In [ ]:
%time data_rescaled = rescale(data_binned)

In [ ]:
plt.figure(figsize = (10, 3.5))

plt.subplot(1, 2, 1)
frame = np.ma.masked_where(~mask, data_binned[:, :, t])
plt.imshow(frame, cmap = "jet")
plt.plot(x, y, '*w', ms = 10)
plt.colorbar()
plt.title("data binned, t = {t}".format(t = t))

plt.subplot(1, 2, 2)
frame = np.ma.masked_where(~mask, data_rescaled[:, :, t])
plt.imshow(frame, cmap = "jet")
plt.plot(x, y, '*w', ms = 10)
plt.colorbar()
plt.title("data rescaled, t = {t}".format(t = t))

plt.show()


fig, ax1 = plt.subplots(figsize = (10, 3.5))
ax1.plot(data_binned[y, x, :], color = "C3")
ax1.set_title("Signal: x = {x}, y = {y}".format(x=x, y=y))
ax1.set_ylabel("data binned", color='C3')

ax2 = ax1.twinx()
ax2.plot(data_rescaled[y, x, :], '.', color = "C4")
ax2.set_ylabel("data rescaled", color='C4')
plt.show()

<a id='section2.5'></a>
### Ensemble averaging

Now let's try to find *cycle length* of our signal and apply ensemble averaging.

In case of known *cycle length* (pacing cycle length (PCL) for example) just use it.

In [ ]:
cycle_length = 75.
signal = data_rescaled[y, x]

plt.figure(figsize = (6, 2))
plt.plot(signal)
plt.plot(np.roll(signal, 75))
plt.show()

So it looks like *cycle_length = 75* is OK.

In [ ]:
%time data_averaged = ensemble_average(data_rescaled, cycle_length)

In [ ]:
plt.figure(figsize = (10, 3.5))

plt.subplot(1, 2, 1)
frame = np.ma.masked_where(~mask, data_rescaled[:, :, t])
plt.imshow(frame, cmap = "jet")
plt.plot(x, y, '*w', ms = 10)
plt.colorbar()
plt.title("data rescaled, t = {t}".format(t = t))

plt.subplot(1, 2, 2)
frame = np.ma.masked_where(~mask, data_averaged[:, :, t])
plt.imshow(frame, cmap = "jet")
plt.plot(x, y, '*w', ms = 10)
plt.colorbar()
plt.title("data averaged, t = {t}".format(t = t))

plt.show()


fig, ax1 = plt.subplots(figsize = (10, 3.5))
ax1.plot(data_rescaled[y, x, :], color = "C4")
ax1.set_title("Signal: x = {x}, y = {y}".format(x=x, y=y))
ax1.set_ylabel("data rescaled", color='C4')

ax2 = ax1.twinx()
ax2.plot(data_averaged[y, x, :], color = "C5")
ax2.set_ylabel("data averaged", color='C5')
plt.show()

<a id='section2.6'></a>
### Transform to phase

In [ ]:
%time phase = transform_to_phase(data_rescaled)

In [ ]:
plt.figure(figsize = (10, 3.5))

plt.subplot(1, 2, 1)
frame = np.ma.masked_where(~mask, data_rescaled[:, :, t])
plt.imshow(frame, cmap = "jet")
plt.plot(x, y, '*w', ms = 10)
plt.colorbar()
plt.title("data rescaled, t = {t}".format(t = t))

plt.subplot(1, 2, 2)
frame = np.ma.masked_where(~mask, phase[:, :, t])
plt.imshow(frame, cmap = "hsv")
plt.plot(x, y, '*w', ms = 10)
plt.colorbar()
plt.title("phase, t = {t}".format(t = t))

plt.show()


fig, ax1 = plt.subplots(figsize = (10, 3.5))
ax1.plot(data_rescaled[y, x, :], color = "C4")
ax1.set_title("Signal: x = {x}, y = {y}".format(x=x, y=y))
ax1.set_ylabel("data rescaled", color='C4')

ax2 = ax1.twinx()
ax2.plot(phase[y, x, :], color = "C6")
ax2.set_ylabel("phase", color='C6')
plt.show()

<a id='section3'></a>
## Mapping

<a id='section3.1'></a>
### APD and Alternance maps

In [ ]:
t_start, t_end = 200, 300
percentage = 60
%time apd_map = calculate_APD_map(data_rescaled, t_start, t_end, percentage).astype(float)

In [ ]:
t_start, t_end = 200, 400
percentage = 60
%time alt_map = calculate_alternance_map(data_rescaled, t_start, t_end, percentage).astype(float)

In [ ]:
plt.figure(figsize = (10, 3.5))

plt.subplot(1, 2, 1)
frame = np.ma.masked_where(~mask, apd_map)
plt.imshow(frame, cmap = "jet")
plt.colorbar(label = "AP duration")
plt.title("APD{percentage} map".format(percentage = percentage))

plt.subplot(1, 2, 2)
frame = np.ma.masked_where(~mask, alt_map)
alt_abs = np.nanmax(np.abs(alt_map))
plt.imshow(frame, cmap = "bwr",
           vmin = -alt_abs, vmax = alt_abs)
plt.colorbar(label = "AP duration difference")
plt.title("Alternance map (for APD{percentage})".format(percentage = percentage))

plt.show()

<a id='section3.2'></a>
### Activation time map

In [ ]:
t_start, t_end = 300, 310
%time act_map = calculate_activation_map(data_rescaled, t_start, t_end, 90.)

In [ ]:
plt.figure(figsize = (1.1 * 3.5, 3.5))
frame = np.ma.masked_where(~mask, act_map)
frame = np.flipud(frame)
plt.contourf(frame, cmap = 'rainbow_r',
             origin = "lower")
plt.colorbar(label = "Activation time", fraction=0.1, pad=0.04)
plt.title("Activation map")
plt.axis('equal')

plt.show()

<a id='section3.3'></a>
### Phase singularity points (PS)

In [ ]:
ps = phase_singularity_detection(phase[:, :, t])

In [ ]:
frame = np.ma.masked_where(~mask, phase[:, :, t])
plt.imshow(frame, cmap = 'hsv')
plt.colorbar()
plt.plot(ps[:, 1], ps[:, 0], 'k.', ms = 10)
plt.plot(ps[:, 1], ps[:, 0], 'w.', ms = 5)
plt.title("PS")
plt.show()